In [41]:
ng_1_data = []
with open("/Users/romanchepenko/Desktop/diplom/duma_analysis/data/duma_years/2022.txt", "r") as read_file:
    for line in read_file:
        ng_1_data.append(line)

In [42]:
pwd

'/Users/romanchepenko/Desktop/diplom/duma_analysis/code'

Сложим все тексты в один список с текстами

In [43]:
full_texts = []
for item in ng_1_data:
    full_texts.append(item)

In [44]:
len(full_texts)

26215

### Предобработка: токенизация, стоп-слова, лемматизация

In [6]:
from nltk.tokenize import word_tokenize 
from nltk import download as nltk_download 
nltk_download ('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/romanchepenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

zsh:1: command not found: wget


In [45]:
#
with open ('stop_ru.txt', 'r') as stop_file:
    rus_stops = [word.strip() for word in stop_file.readlines()]

In [46]:
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'

<>:1: DeprecationWarning: invalid escape sequence \]
<>:1: DeprecationWarning: invalid escape sequence \]
/var/folders/56/yw0nw4q15bzf2bfcm7wgd74w0000gn/T/ipykernel_19215/1721479160.py:1: DeprecationWarning: invalid escape sequence \]
  punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


In [47]:
filter = rus_stops + list (punctuation)

In [48]:
filter

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [49]:
# Добавляем ненужные для анализа слова в список стопслов
filter.extend(['это', 'нею', 'который', 'чел', 'такой', 'дать', 'самый', 'должный', 'знать', 'давать', 'вообще', 'какой-то', 'понимать', 'один', '…', 'что', 'поэтому', 'далее', 'думать', 'всё-таки', 'вместо', '0', '0,0', 'час', 'мина', 'в.', 'сечь'])

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 8.2MB 10.4MB/s 


In [19]:
from pymorphy2 import MorphAnalyzer
parser = MorphAnalyzer ()

In [50]:
def preprocess (input_text):
    '''функция для предобработки текста'''
    ## токенизируем через nltk:
    tokenized_text = word_tokenize (input_text.lower())
    ## убираем пунктуацию и стоп-слова:
    clean_text = [word for word in tokenized_text if word not in filter]
    ## лемматизируем pymorphy2:
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]
    return lemmatized_text

In [51]:
%%time
tokenized_and_preprocessed_texts = [preprocess (text) for text in full_texts]

CPU times: user 49.2 s, sys: 77.5 ms, total: 49.3 s
Wall time: 49.3 s


Посмотрим, как оно выглядит в предобработанном виде:

In [52]:
for text in tokenized_and_preprocessed_texts [0:3]:
  print (text)

['18', 'январь', '2022']
[]
[]


### Теперь можно делать собственно топик моделинг:

In [25]:
import gensim

In [24]:
!pip install gensim

     |████████████████████████████████| 24.0 MB 14.9 MB/s eta 0:00:01   |█▏                              | 901 kB 2.2 MB/s eta 0:00:11
     |████████████████████████████████| 58 kB 15.2 MB/s eta 0:00:01


Список токенизированных и лемматизированных файлов надо превратить в специальный объект `gensim.corpora.Dictionary`:

In [53]:
gensim_dictionary_for_TM = gensim.corpora.Dictionary(tokenized_and_preprocessed_texts)

Метод .filter_extremes удаляет из нашего корпуса слова, которые 
* встречаются **более** чем в каком-то **проценте** от документов нашего корпуса (параметр `no_above`, ниже он стоит 10%)
* встречаются **менее** чем в каком-то **числе** документов корпуса
(параметр `no_below`, ниже он стоит 20 документов):

In [54]:
gensim_dictionary_for_TM.filter_extremes(no_above=0.1, no_below=20)

После `filter_extremes` делают `.compactify()`, чтобы немножко сжать ваш генсим-словарь: в нем удалено много слов и освободились какие-то id. Значит, можно перераспредилить id, чтобы в среднем они стали покороче и занимали меньше места: 

In [55]:
gensim_dictionary_for_TM.compactify()

In [56]:
print(gensim_dictionary_for_TM)

Dictionary<2461 unique tokens: ['18', '2022', 'январь', '12', 'большой']...>


Преобразуем наши тексты в мешки слов с помощью встроенного в генсимовский словарь метода `.doc2bow` (т.е. document to bag-of-words)

In [57]:
corpus = [gensim_dictionary_for_TM.doc2bow(text) for text in tokenized_and_preprocessed_texts]

Все, теперь можно делать топик моделинг:

In [58]:
%%time
lda = gensim.models.LdaMulticore(corpus, 
                                 num_topics = 40, # число топиков
                                 id2word=gensim_dictionary_for_TM, 
                                 passes=5)

CPU times: user 1.99 s, sys: 460 ms, total: 2.45 s
Wall time: 9.67 s


In [59]:
lda.print_topics()

[(10,
  '0.034*"комитет" + 0.024*"закон" + 0.021*"государственный" + 0.016*"проект" + 0.016*"дума" + 0.015*"федеральный" + 0.012*"федерация" + 0.011*"российский" + 0.009*"уважаемый" + 0.008*"чтение"'),
 (8,
  '0.019*"решение" + 0.015*"евгений" + 0.013*"животное" + 0.011*"наш" + 0.009*"принять" + 0.009*"говорить" + 0.007*"сергеевич" + 0.006*"регион" + 0.006*"иванович" + 0.006*"который"'),
 (22,
  '0.033*"банк" + 0.020*"система" + 0.018*"центральный" + 0.011*"уважаемый" + 0.009*"эльвира" + 0.009*"сахипзадович" + 0.009*"работа" + 0.008*"местный" + 0.008*"наш" + 0.008*"рынок"'),
 (3,
  '0.040*"выступить" + 0.038*"желать" + 0.024*"слово" + 0.017*"президент" + 0.013*"..." + 0.012*"хотеть" + 0.011*"правительство" + 0.010*"мотив" + 0.009*"работа" + 0.009*"заключительный"'),
 (2,
  '0.132*"фракция" + 0.096*"россия" + 0.070*"единый" + 0.044*"а." + 0.023*"справедливый" + 0.020*"правда" + 0.019*"с." + 0.016*"н." + 0.015*"г." + 0.013*"председатель"'),
 (0,
  '0.023*"наш" + 0.013*"страна" + 0.012*"р

In [60]:
lda.print_topics (-1)

[(0,
  '0.023*"наш" + 0.013*"страна" + 0.012*"российский" + 0.011*"федерация" + 0.011*"гражданин" + 0.010*"хотеть" + 0.008*"поддержать" + 0.008*"должный" + 0.007*"культура" + 0.007*"правительство"'),
 (1,
  '0.013*"орган" + 0.010*"данный" + 0.009*"федеральный" + 0.009*"власть" + 0.009*"закон" + 0.009*"субъект" + 0.008*"правительство" + 0.008*"правило" + 0.008*"система" + 0.008*"..."'),
 (2,
  '0.132*"фракция" + 0.096*"россия" + 0.070*"единый" + 0.044*"а." + 0.023*"справедливый" + 0.020*"правда" + 0.019*"с." + 0.016*"н." + 0.015*"г." + 0.013*"председатель"'),
 (3,
  '0.040*"выступить" + 0.038*"желать" + 0.024*"слово" + 0.017*"президент" + 0.013*"..." + 0.012*"хотеть" + 0.011*"правительство" + 0.010*"мотив" + 0.009*"работа" + 0.009*"заключительный"'),
 (4,
  '0.031*"подготовиться" + 0.016*"строительство" + 0.012*"михаилович" + 0.009*"работать" + 0.009*"андрей" + 0.007*"неделя" + 0.007*"жильё" + 0.007*"макаров" + 0.007*"программа" + 0.007*"дорога"'),
 (5,
  '0.128*"поправка" + 0.078*"табл

Основные параметры, которыми мы можем управлять, это num_topics и passes.

**num_topics** - это количество тем. Его обычно подбирают либо на глаз, глаяда на то, насколько хорошо интерпретируемыми получаются темы, либо с использованием специальных метрик типа 'перплексии', которые оценивают однородность топиков автоматически. По умолчанию в генсиме он 100.  

**passes** - задает количество проходов по данным. Чем больше, тем лучше сойдется модель, но обучаться будет дольше.

Про параметры можно почитать в документации:

In [34]:
?gensim.models.LdaMulticore

Визуализация:

Вроде бы можно сделать с помощью pyLDAvis — специальной библиотеки для визуализации результатов LDA в питоне. 

In [35]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 2.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=ca1d7458057d75ccc6dfe69d10fc958f69aa3978396c95c134154fc5eaf998da
  Stored in directory: /Users/romanchepenko/Library/Caches/pip/wheels/57/a4/86/d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=305c5563eef3e0642f8cb304181c69e337b172cce2bffcbbad358f490bb1e1dd
  Stored in directory: /Users/romanchepenko/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built pyldavis sklearn


In [36]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [61]:
vis = gensimvis.prepare(lda, corpus, gensim_dictionary_for_TM)

/Users/romanchepenko/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/romanchepenko/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [62]:
pyLDAvis.enable_notebook()

In [63]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
37    -0.099177 -0.057863       1        1  9.796369
21    -0.086298 -0.087947       2        1  7.163750
38     0.047367  0.112312       3        1  5.944478
19    -0.065027 -0.015257       4        1  4.007690
35     0.155032  0.140617       5        1  3.906170
0     -0.083035 -0.032815       6        1  3.801193
22    -0.076168 -0.022029       7        1  3.467394
10     0.021555  0.113238       8        1  3.149951
27    -0.003065  0.090589       9        1  3.105990
1     -0.026969 -0.007129      10        1  2.776479
2      0.074544  0.160278      11        1  2.718743
32    -0.079965 -0.027530      12        1  2.392362
29    -0.044614 -0.054559      13        1  2.363675
3     -0.033990 -0.018742      14        1  2.343828
4     -0.058658 -0.045044      15        1  2.248594
25    -0.064791 -0.019071      16        1  2.243487
9     -0.063367 -0.064479      17        1  2.199148
12    -0.062350 -0.037020      18        1  2.181946
11    -0.031681 -0.037825      19        1  2.113419
30    -0.032222  0.043220      20        1  2.033010
34    -0.050005  0.040665      21        1  2.009726
8     -0.047849  0.006440      22        1  1.930189
26     0.047763  0.137884      23        1  1.928626
36    -0.038449  0.013758      24        1  1.922071
23     0.018786  0.078333      25        1  1.898576
5      0.185734  0.078642      26        1  1.656710
20    -0.066096 -0.050376      27        1  1.644929
15    -0.003556 -0.065291      28        1  1.581903
7      0.031782 -0.006146      29        1  1.576107
13     0.216750 -0.187432      30        1  1.574716
17    -0.046995 -0.027455      31        1  1.567422
39    -0.064845 -0.050518      32        1  1.517692
33    -0.021894  0.022056      33        1  1.421505
6     -0.029455  0.009679      34        1  1.417433
18     0.303929 -0.223812      35        1  1.380962
16     0.015175  0.103207      36        1  1.126109
14    -0.061400 -0.011837      37        1  1.106024
31     0.146901  0.028624      38        1  1.020194
28     0.065807  0.000245      39        1  0.945870
24     0.010796 -0.029611      40        1  0.815561, topic_info=                      Term         Freq        Total Category  logprob  \
1167           голосование  2135.000000  2135.000000  Default  30.0000   
1157            голосовать  1064.000000  1064.000000  Default  29.0000   
15    председательствующий  2234.000000  2234.000000  Default  28.0000   
1171         проголосовать  1316.000000  1316.000000  Default  27.0000   
57                 фракция  1926.000000  1926.000000  Default  26.0000   
...                    ...          ...          ...      ...      ...   
10            председатель    15.399624   662.645696  Topic40  -5.1186   
471                    ...    16.729375  1184.789039  Topic40  -5.0358   
63                     наш    15.287858  1960.189074  Topic40  -5.1259   
672             михаилович    10.945374   469.858071  Topic40  -5.4600   
1171         проголосовать    10.601933  1316.223715  Topic40  -5.4919   

      loglift  
1167  30.0000  
1157  29.0000  
15    28.0000  
1171  27.0000  
57    26.0000  
...       ...  
10     1.0472  
471    0.5489  
63    -0.0447  
672    1.0495  
1171  -0.0124  

[2230 rows x 6 columns], token_table=      Topic      Freq Term
term                      
525       1  0.063353   ..
525       2  0.050682   ..
525       4  0.076024   ..
525       7  0.088694   ..
525      10  0.025341   ..
...     ...       ...  ...
1265     34  0.021049    №
1265     36  0.018042    №
1265     37  0.003007    №
1265     39  0.009021    №
1265     40  0.003007    №

[23232 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[38, 22, 39, 20, 36, 1, 23, 11, 28, 2, 3, 33, 30, 4, 5, 26, 10, 13, 12, 31, 35, 9, 27, 37, 24, 6, 21, 16, 8, 14, 18, 40, 34, 7, 19, 17, 15, 32, 29, 25])

In [64]:
# Сохраняем график в отдельный html файл
pyLDAvis.save_html(vis, 'lda_2022.html')